In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt

import torch, torchvision
import torchvision.transforms as T
from torch import nn
from torch import optim
from torchvision.models import resnet50
import torch.nn.functional as F
from torch.utils.data.dataloader import default_collate

sys.path.append("./libs/ACDA")
import Conv_DCFD as acda
from crowd import Crowd_sh

### Crowd counting
The paper uses ACDA trained with ImageNet as feature extractor and replace final linear layer with few standard transposed convolutional layers for recovering 

Note: wanted to use ShanghaiTech subset A dataset since it is smaller than UCF-QNRF.
Download link:
https://www.kaggle.com/datasets/tthien/shanghaitech

Paper that provided download link and the processing of the ShanghaiTech dataset (need to add citation to paper!)
https://github.com/cvlab-stonybrook/DM-Count

For the crowd counting need to implement Resnet-18 (Ad-ResNet-s listed in Table A of appendix) but with out the first "max pool" layer and final "average pool" layer.
The Ad-ResNet-s network was very similar to ResNet-18, so the code for a ResNet-18 was used as a starting point and updated with the changes described in Appendix A.4 for crodwd counting.
Reference for how to implement ResNet-18:
https://www.kaggle.com/code/ivankunyankin/resnet18-from-scratch-using-pytorch/notebook

ResNet original paper: https://arxiv.org/pdf/1512.03385.pdf


In [3]:
# base code for ResNet18 came from link below, modified based on Appendix A structure
# https://www.kaggle.com/code/ivankunyankin/resnet18-from-scratch-using-pytorch?scriptVersionId=54460301&cellId=23

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, kernel_size_acda = None, identity_downsample=None, stride=1, acda_padding=3):
        super(Block, self).__init__()
        self.kernel_size_acda = kernel_size_acda
        if kernel_size is None:
            kernel_size = 3
        if kernel_size_acda:
            self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, stride=stride, padding=1)
            self.bn1 = nn.BatchNorm2d(in_channels)
            self.acda_conv = acda.Conv_DCFD(in_channels, in_channels, kernel_size=kernel_size_acda, stride=1, padding=acda_padding)
            self.acda_bn = nn.BatchNorm2d(in_channels)
            self.conv2 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(out_channels)
        else:
            self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=1)
            self.bn1 = nn.BatchNorm2d(out_channels)
            self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=kernel_size, stride=1, padding=1)
            self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample

    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        if self.kernel_size_acda:
            x = self.acda_conv(x)
            x = self.acda_bn(x)
            x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

class ResNet_18(nn.Module):
    def __init__(self, image_channels):
        super(ResNet_18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        
        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2)
        self.layer4 = self.__make_layer(256, 512, stride=2)

    def __make_layer(self, in_channels, out_channels, stride, temp_padding=1):
        identity_downsample = None
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels, temp_padding)
            
        return nn.Sequential(
            Block(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride), 
            Block(out_channels, out_channels)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x 

    def identity_downsample(self, in_channels, out_channels, temp_padding):   
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=temp_padding), 
            nn.BatchNorm2d(out_channels)
        )

class ResNet_18_ACDA(nn.Module):
    def __init__(self, image_channels):
        super(ResNet_18_ACDA, self).__init__()    
        self.in_channels = 64
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        
        #resnet layers
        self.layer1 = self.__make_layer(64, 64, stride=1)
        self.layer2 = self.__make_layer(64, 128, stride=2)
        self.layer3 = self.__make_layer(128, 256, stride=2, kernel_size=1, kernel_size_acda=7, temp_padding=4)
        self.layer4 = self.__make_layer(256, 512, stride=2, kernel_size=1, kernel_size_acda=5, acda_padding=2, temp_padding=4)
        
    def __make_layer(self, in_channels, out_channels, stride, kernel_size=None, kernel_size_acda=None, temp_padding=1, acda_padding=3):
        identity_downsample = None
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels, temp_padding)

        return nn.Sequential(
            Block(in_channels, out_channels, identity_downsample=identity_downsample, 
                  stride=stride, kernel_size=kernel_size, kernel_size_acda=kernel_size_acda, acda_padding=acda_padding), 
            Block(out_channels, out_channels)
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x 
    
    def identity_downsample(self, in_channels, out_channels, temp_padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=temp_padding), 
            nn.BatchNorm2d(out_channels)
        )

resNet = ResNet_18(3)
resNet_acda = ResNet_18_ACDA(3)

In [4]:
# Train network
mse_loss = nn.MSELoss()

def train(model, optimizer, train_data, num_epochs=2):
    mae_vals = []
    mse_vals = []
    loss_vals = []
    for _ in range(num_epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        mae = 0
        mse = 0
        count = 0
        for i, (inputs, points, gt_discrete) in enumerate(train_data):
            # get the inputs
            gd_count = np.array([len(p) for p in points], dtype=np.float32)
            N = inputs.size(0)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            # print(outputs.shape, outputs)
            loss = mse_loss(outputs.sum(1).sum(1).sum(1), torch.from_numpy(gd_count).float())
            loss.backward()
            optimizer.step()

            pred_count = torch.sum(outputs.view(N, -1), dim=1).detach().cpu().numpy()
            pred_err = pred_count - gd_count

            count += N
            mae += np.mean(abs(pred_err)) * N
            mse += np.mean(pred_err * pred_err) * N

            # print statistics
            loss_vals.append(loss.item() / N)
            running_loss += loss.item()
            if i % 10 == 9:    # print every 2000 mini-batches
                # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                loss_vals.append(running_loss / 10)
                running_loss = 0.0
        mae_vals.append(mae * 1.0 / count)
        mse_vals.append(mse * 1.0 / count)
    return loss_vals, mae_vals, mse_vals


In [5]:
def train_collate(batch):
    transposed_batch = list(zip(*batch))
    images = torch.stack(transposed_batch[0], 0)
    points = transposed_batch[1]  # the number of points is not fixed, keep it as a list of tensor
    gt_discretes = torch.stack(transposed_batch[2], 0)
    return images, points, gt_discretes

# load the data
train_path = './data/ShanghaiTech/part_A/train_data/'
test_path = './data/ShanghaiTech/part_A/test_data/'

train_crowd = Crowd_sh(train_path, crop_size=112, method='train') # YVETTE - should that be 512 instead...?
test_crowd = Crowd_sh(test_path, crop_size=112, method='val')

# batch_size = 10
batch_size = 64
trainloader = torch.utils.data.DataLoader(train_crowd, batch_size=batch_size,
                                          shuffle=True, num_workers=0,
                                          collate_fn=train_collate,
                                          pin_memory=True)

testloader = torch.utils.data.DataLoader(test_crowd, batch_size=batch_size,
                                         shuffle=False, num_workers=0,
                                         collate_fn=default_collate,
                                         pin_memory=False)

number of img: 300
number of img: 182


In [22]:
resNet_optimizer = optim.SGD(resNet.parameters(), lr=0.001, momentum=0.9)
resNet_acda_optimizer = optim.SGD(resNet_acda.parameters(), lr=0.001, momentum=0.9)

# ResNet
epochs = 4
loss, mae, mse = train(resNet, resNet_optimizer, trainloader, epochs)

# ResNet + ACDA
loss_acda, mae_acda, mse_acda = train(resNet_acda, resNet_acda_optimizer, trainloader, epochs)


In [23]:
resNet_params = sum(p.numel() for p in resNet.parameters() if p.requires_grad)
resNet_acda_params = sum(p.numel() for p in resNet_acda.parameters() if p.requires_grad)

print('ResNet-18')
print(f'Parameters = {resNet_params}, MAE = {sum(mae)/len(mae)}, MSE = {sum(mse)/len(mse)}\n')

print('Ad-ResNet-s')
print(f'Parameters = {resNet_acda_params}, MAE = {sum(mae_acda)/len(mae_acda)}, MSE = {sum(mse_acda)/len(mse_acda)}')


ResNet-18
Parameters = 12557568, MAE = 16.226666673024496, MSE = 875.3816657511393

Ad-ResNet-s
Parameters = 9196956, MAE = 16.384166707992556, MSE = 934.4541670735678
